![img](https://www.kdnuggets.com/wp-content/uploads/text-preprocessing-framework-2.png)

In [1]:
! pip install contractions

In [2]:
! pip install inflect

In [3]:
! pip install wget

In [1]:
import pandas as pd
import numpy as np
import nltk.data
from nltk import word_tokenize, sent_tokenize
from sklearn.feature_extraction.text import CountVectorizer
from nltk.corpus import stopwords
import gensim
from gensim.models import Word2Vec
from gensim.models import FastText
import contractions
import inflect
import re, unicodedata
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.layers import Embedding
from keras.models import Model
from keras.layers import Input, Flatten, Dense

#import my_functions

pd.set_option('max_colwidth', 250)

C:\Users\Enric\Anaconda3\lib\site-packages\gensim\utils.py:1197: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [2]:
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Enric\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Enric\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [3]:
STOPWORDS_ESP = list(set(stopwords.words('spanish')))
STOPWORDS_CAT = open('stopwords_catalan.txt', 'r', encoding= 'UTF-8').read().split()
STOPWORDS_ENG = list(set(stopwords.words('english')))
CUSTOM  = ['uf', '100st', '100t', '100u', 'pal', 'zzb21601xu', 'x2']

STOPWORDS_ALL = STOPWORDS_ESP + STOPWORDS_CAT + STOPWORDS_ENG + CUSTOM

In [4]:
import wget

print('Beginning file download with wget module')

url = 'https://raw.githubusercontent.com/michmech/lemmatization-lists/master/lemmatization-es.txt'  
#wget.download(url) 

Beginning file download with wget module


In [5]:
lemmatizer = {}
with open("lemmatization-es.txt", encoding= 'UTF-8') as f:
    for line in f:
        try:
            (key, val) = line.split()
        except:
            pass
        lemmatizer[str(key)] = val

In [6]:
lemmatizer_inv = {v: k for k, v in lemmatizer.items()}

In [7]:
#lemmatizer_inv

In [8]:
data = pd.read_csv('../02_Comprension de Datos/Descripciones204k_20190315.csv', sep = ';')

In [9]:
data.head()

,id,brand,text
0,1060651400131,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo masculino con textura de mujer
1,1060651400180,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo doble faz de mujer con cinturón a tono
2,1051056400107,Woman_El_Corte_Inglés,Moda Mujer Abrigos Abrigo largo de antelina de mujer Woman El Corte Inglés
3,1019350401147,Lloyd's,Moda Mujer Abrigos Chaqueta térmica de mujer Lloyds con efecto cortavientos
4,1019353400229,Lloyd's,Moda Mujer Abrigos Parka 100% algodón de mujer Lloyds con capucha


Creamos una lista de marcas para poder quitarlas del texto en caso de que aparezcan, eliminamos las marcas duplicadas para comprimir el tamaño de la lista y optimizar el proceso de bsuqueda

In [10]:
copia = data.copy()

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def normalize_brands(words):
    words = str(words)
    words  = words.strip().split()
    words = to_lowercase(words)
    words = remove_punctuation(words)    
    words = list(dict.fromkeys(words)) #Remove duplicates
    words = " ".join(words)
    return words
    
copia['brand'] = copia['brand'].apply(normalize_brands)
brands = list(dict.fromkeys(copia['brand'].values))
brands.remove('marvel')
more_brands = ['tarocco', 'tucci', 'fjord', 'tommy', 'hilfiger', 'easy', 'wear', 'josma']
for brand in more_brands:
    brands.append(brand)

In [11]:
len(brands)

5088

## Normalizacion

Con la normalización vamos a realizar una serie de tareas destinadas a poner todo el texto en un campo de juego nivelado: convertir todo el texto a la misma mayúscula o minúscula, eliminar la puntuación, convertir las cifras a sus equivalentes en palabras, etc. La normalización pone todas las palabras en pie de igualdad y permite que el procesamiento se realice de manera uniforme. Algunas de las técnicas que vamos a aplicar son:

- Borrar caracteres extraños
- Pasar todo el texto a minusculas
- Quitar simbolos de puntuacion ( . , &, !, ?, ¿, /, etc)
- Pasar numeros en digito a texto
- Quitar stopwords
- Stemming
- Lexematizar

**Importante:** Despues de la Nomralizacion trabajaremos a nivel de palabra o token en vez de a nivel de texto

In [12]:
ngrams = ['corte ingles', 'bob esponja', 'color azul', 'cristiano ronaldo', 'fc barcelona', 'leo messi', 'real madrid',
         ]

In [13]:
def remove_non_ascii(words):
    """Remove non-ASCII characters from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = unicodedata.normalize('NFKD', word).encode('ascii', 'ignore').decode('utf-8', 'ignore')
        #new_word = unicodedata.normalize('NFD', word).encode('ascii', 'ignore')
        new_words.append(new_word)
    return new_words

def to_lowercase(words):
    """Convert all characters to lowercase from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = word.lower()
        new_words.append(new_word)
    return new_words

def remove_punctuation(words):
    """Remove punctuation from list of tokenized words"""
    new_words = []
    for word in words:
        new_word = re.sub(r'[^\w\s]', '', word)
        if new_word != '':
            new_words.append(new_word)
    return new_words

def replace_numbers(words):
    """Remove all interger occurrences in list"""
    new_words = []
    for word in words:
        if not word.isdigit():
            new_words.append(word)
    return new_words

def remove_stopwords(words):
    """Remove stop words from list of tokenized words"""
    new_words = []
    for word in words:
        if word not in STOPWORDS_ALL and len(word) > 1:
            new_words.append(word)
    return new_words

def stem_words(words):
    """Stem words in list of tokenized words"""
    stemmer = SnowballStemmer('spanish')
    stems = []
    for word in words:
        stem = stemmer.stem(word)
        stems.append(stem)
    return stems

def remove_brands(words):
    """Remove brands from the text"""
    clean_words = []
    for word in words:
        if word not in brands:
            clean_words.append(word)
    return clean_words

def lemmatize(words):
    """Lemmatize verbs in list of tokenized words"""
    lemmas = []
    for word in words:
        if word in lemmatizer_inv:
            lemmas.append(lemmatizer_inv[word])
        else: lemmas.append(word)
    return lemmas

def replace_ngrams(words):
    for gram in ngrams:
        if (gram in words):
            g = "_".join(gram.split())
            words = words.replace(gram, g)
    return words

def normalize(words):
    #if pandas == True:
    words  = words.strip().split()
    words = to_lowercase(words)
    words = remove_punctuation(words)
    words = remove_brands(words)
    words = remove_stopwords(words)
    words = lemmatize(words)
    words = replace_numbers(words)
    words = remove_non_ascii(words)
    words = list(dict.fromkeys(words)) #Remove duplicates
    words = " ".join(words)
    words = replace_ngrams(words)
    return words


In [14]:
data_copy = data.copy()

In [15]:
%%time
#4min 4sec
data_copy['text'] = data_copy['text'].apply(normalize)

Wall time: 11min 32s


In [16]:
data_copy.head(5)

,id,brand,text
0,1060651400131,Woman_Limited_El_Corte_Inglés,moda mujer abrigo masculino textura
1,1060651400180,Woman_Limited_El_Corte_Inglés,moda mujer abrigo doble faz cinturon tono
2,1051056400107,Woman_El_Corte_Inglés,moda mujer abrigo largo antelina woman corte_ingles
3,1019350401147,Lloyd's,moda mujer abrigo chaqueta termica efecto cortavientos
4,1019353400229,Lloyd's,moda mujer abrigo parka algodon capucha


In [17]:
data.head(5)

,id,brand,text
0,1060651400131,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo masculino con textura de mujer
1,1060651400180,Woman_Limited_El_Corte_Inglés,Moda Mujer Abrigos Abrigo doble faz de mujer con cinturón a tono
2,1051056400107,Woman_El_Corte_Inglés,Moda Mujer Abrigos Abrigo largo de antelina de mujer Woman El Corte Inglés
3,1019350401147,Lloyd's,Moda Mujer Abrigos Chaqueta térmica de mujer Lloyds con efecto cortavientos
4,1019353400229,Lloyd's,Moda Mujer Abrigos Parka 100% algodón de mujer Lloyds con capucha


In [18]:
#7000
data_copy.loc[200000]

id                                                      1005112393706
brand                                                          ISLAND
text     musica dance electronica ambient chill omen remixes lpvinilo
Name: 200000, dtype: object

In [19]:
data_copy['text'].shape

(204812,)

In [20]:
maxw = 0
mean = 0
max_s = 0
for i, sentence in enumerate(data_copy['text']):
    mean += len(sentence.split())
    if maxw < len(sentence.split()):
        maxw = len(sentence.split())
        max_s = i
print('Max words in a sentence:' + ' '*10 + 'Mean words in a sentence:')
print("-" * 60)
print (f"{maxw} {' '*30} {mean/len(data_copy['text'])}")
print("-" * 60)
print (data_copy['text'][max_s])

Max words in a sentence:          Mean words in a sentence:
------------------------------------------------------------
23                                8.072564107571822
------------------------------------------------------------
libro salud bienestar dieta nutricion general detox sen sano dentro bello clave nutricional rutina diarias eliminar toxina forma saludable energetica nutritiva tapa blanda


In [21]:
data_copy.to_csv('Texto_PreProcesado.csv', sep = ';', index = False)

### N-gramas (CountVectorizer)

In [22]:
bag_of_words = nltk.word_tokenize(data_copy['text'].to_string())
bag_of_words = list(dict.fromkeys(bag_of_words))
print(bag_of_words[:10])
print(len(bag_of_words)) 
#126k productos = 2_063_845 palabras
#175k productos = 2_838_097 palabras
#204k productos = 3_070_773 palabras -> 1_878_308 20/03/2019 -> no dups = 257042, ngrams =257044 

['0', 'moda', 'mujer', 'abrigo', 'masculino', 'textura', '1', 'doble', 'faz', 'cinturon']
257052


In [23]:
corpus = [sent for sent in data_copy['text']]
corpus[0:5]

['moda mujer abrigo masculino textura',
 'moda mujer abrigo doble faz cinturon tono',
 'moda mujer abrigo largo antelina woman corte_ingles',
 'moda mujer abrigo chaqueta termica efecto cortavientos',
 'moda mujer abrigo parka algodon capucha']

In [24]:
vectorizer = CountVectorizer(ngram_range=(1, 2))
X = vectorizer.fit_transform(corpus)

In [25]:
list(vectorizer.vocabulary_.keys())[0:10]

['moda',
 'mujer',
 'abrigo',
 'masculino',
 'textura',
 'moda mujer',
 'mujer abrigo',
 'abrigo masculino',
 'masculino textura',
 'doble']

In [26]:
import pprint
#pprint.pprint (vectorizer.vocabulary_)

---

## Word2Vec


In [27]:
sentences = [word.split() for word in data_copy['text'].values]
sentences[:3]

[['moda', 'mujer', 'abrigo', 'masculino', 'textura'],
 ['moda', 'mujer', 'abrigo', 'doble', 'faz', 'cinturon', 'tono'],
 ['moda', 'mujer', 'abrigo', 'largo', 'antelina', 'woman', 'corte_ingles']]

In [28]:
len(sentences)

204812

In [29]:
%%time
model = Word2Vec(sentences, workers = 3, min_count=5, window = 10, size = 200)
model.train(sentences, total_examples=len(sentences), epochs=50)

Wall time: 3min 38s


In [30]:
#model.save("word2vec_model_v1")

In [32]:
#model = Word2Vec.load("word2vec_model_v1")
wl = 'futbol'
model.wv.most_similar (positive = wl)
#model.wv.most_similar_cosmul(positive = ['disfraz', 'abrigo'])

[('baloncesto', 0.44440093636512756),
 ('fc_barcelona', 0.4238649010658264),
 ('madrid', 0.39621978998184204),
 ('futboltaco', 0.3959360122680664),
 ('cf', 0.3918527364730835),
 ('equipacion', 0.3738066852092743),
 ('atletico', 0.370324969291687),
 ('escudo', 0.3675953447818756),
 ('real_madrid', 0.3640595078468323),
 ('turf', 0.3561278283596039)]

In [142]:
def similar_products(text):
    text = normalize(text)
    list_text = text.split()
    most_similar = model.wv.most_similar_cosmul(positive = list_text)
    
    return most_similar
    
similar_products('Silla de paseo')

[('carrycot', 0.4795910120010376),
 ('cambiador', 0.4572114944458008),
 ('portabebes', 0.44224661588668823),
 ('maternal', 0.43625256419181824),
 ('trona', 0.4313957989215851),
 ('duet', 0.4276811182498932),
 ('auto', 0.4258055090904236),
 ('capazo', 0.425559937953949),
 ('isofix', 0.42419910430908203),
 ('modular', 0.42381948232650757)]

---

## Vectorize Sentences

- Initialize tokenizer with num_words = MAX_NB_WORDS (200K). i.e. The tokenizer will perform a word count, sorted by number of occurences in descending order and pick top N words, 200K in this case 
- Use tokenizer's texts_to_sequences method to convert text to array of integers.
- The arrays obtained from previous step might not be of uniform length, use pad_sequences method to obtain arrays with length equal to MAX_SEQUENCE_LENGTH (30)

In [148]:
MAX_NB_WORDS = 257053 #200k
MAX_SEQUENCE_LENGTH = 23                                

In [149]:
all_text = data_copy['text']
all_text = all_text.drop_duplicates (keep = False)

tokenizer = Tokenizer(num_words=MAX_NB_WORDS)
tokenizer.fit_on_texts(all_text)

data_sequences = tokenizer.texts_to_sequences(data_copy['text'])
data_vec = pad_sequences(data_sequences, maxlen=MAX_SEQUENCE_LENGTH)

In [150]:
word_index = tokenizer.word_index
print('Found %s unique tokens.' % len(word_index))

Found 49422 unique tokens.


In [151]:
data_vec[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    1,    3,   94, 2461,
       1432])

A word_index has a unique ID assigned to each word in the data. Example:

In [152]:
word_index = tokenizer.word_index
test_string = "ropa deporte abrigo raqueta bebe"
print("word\t\tid")
print("-" * 20)
for word in test_string.split():
    print("%s\t\t%s" % (word, word_index[word]))

word		id
--------------------
ropa		34
deporte		12
abrigo		94
raqueta		1644
bebe		4


In [153]:
EMBEDDING_DIM=200

In [166]:
word_vectors = model.wv
vocabulary_size = min(len(word_index)+1,MAX_NB_WORDS)
embedding_matrix = np.zeros((vocabulary_size, EMBEDDING_DIM))

for word, i in word_index.items():
    if i>=MAX_NB_WORDS:
        continue
    try:
        embedding_vector = word_vectors[word]
        embedding_matrix[i] = embedding_vector
    except KeyError:
        embedding_matrix[i]=np.random.normal(0,np.sqrt(0.25),EMBEDDING_DIM)

del(word_vectors)

embedding_layer = Embedding(input_dim = vocabulary_size,
                            output_dim = EMBEDDING_DIM,
                            input_length = MAX_SEQUENCE_LENGTH,
                            weights=[embedding_matrix],
                            name='w2v_embedding',
                            trainable=False)

In [167]:
model.save("w2v_embedding_v1.h5")

In [174]:
i = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32', name='main_input')
x = embedding_layer(i)
x = Flatten()(x)
o = Dense(1, activation='sigmoid')(x)

model = Model(inputs=i, outputs=o)

In [175]:
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
main_input (InputLayer)      (None, 23)                0         
_________________________________________________________________
w2v_embedding (Embedding)    (None, 23, 200)           9884600   
_________________________________________________________________
flatten_2 (Flatten)          (None, 4600)              0         
_________________________________________________________________
dense_1 (Dense)              (None, 1)                 4601      
Total params: 9,889,201
Trainable params: 4,601
Non-trainable params: 9,884,600
_________________________________________________________________
None
